## **Automatique validation**



#### **1- Connect to your instance with the sdk**

In [1]:
from toucan_data_sdk import ToucanDataSdk
import pandas as pd
import getpass

instance = 'scc'
small_app = 'lucent'
instance_url = f"https://api-{instance}.toucantoco.com"
username = 'toucantoco'
try:
    auth = get_auth(instance)
except Exception:
    auth = (username, getpass.getpass())
    
sdk = ToucanDataSdk(instance_url, small_app=small_app, auth=auth)

In [2]:
import numpy as np
import json

#### **2- Get the datasources and etl**

In [11]:
# You can choose specific ouput domains
data_sources_dict = sdk.get_datasources()
etl = sdk.get_etl()


#### **3- update etl**

In [15]:
def create_validation_array(data_source):
    """
    Buidl validation array with only two steps of validation:
        - type: 'data_type'
        - type: 'columns'
    """
    
    validation = [
        {'type': 'columns'},
        {'type': 'data_type'}
    ]
    
    #step 1:
    expected = [c for c in list(data_source.columns) if not c.startswith('_')]
    validation[0]['expected'] = expected
    
    # step 2 : 
    expected = dict(
        data_source.dtypes.apply(lambda x: 'number' if np.issubdtype(x, np.number) else 'string')
    )
    validation[1]['expected'] = expected
    return validation

In [16]:
if __name__ == 'ipython':
    for datasources_conf in etl['DATA_SOURCES']:
        data_source = data_sources_dict[datasources_conf['domain']]
        validation = create_validation_array(data_source)

        # Ce que je fais ci apres est ameilliorable... Des feedbacks ?
        expected = validation[1]['expected']
        expected_corrected = {}
        for col_name, dtype in expected.items():
            if col_name.startswith('_'):
                print(col_name)
                continue
            if dtype == 'number':
                expected_corrected[col_name] = dtype
            if dtype == 'string':
                try:
                    if not data_source[col_name].astype(str).str.replace('.','').str.replace(',','').str.isdigit().any():
                        expected_corrected[col_name] = dtype
                except Exception:
                    pass
        # --

        validation[1]['expected'] = expected_corrected
        datasources_conf['validation'] = validation

__filename__
__filename__
__filename__


In [17]:
requete = sdk.client.config.etl.put(json = etl)
if requete.status_code == 200:
    print('oh yeah! ... go check if validation pass now :)')

oh yeah! ... go check if validation pass now :)
